In [1]:
import cv2
import easyocr
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.117 🚀 Python-3.9.13 torch-2.0.1+cu117 CPU
Setup complete ✅ (4 CPUs, 3.7 GB RAM, 77.7/233.2 GB disk)


In [3]:
from ultralytics import YOLO

from IPython.display import display, Image

In [4]:
# load an pretrained model
model = YOLO('best.pt') 
model.conf = 0.4  #confidence threshold for detection

In [5]:
def preprocess_image(image):
    # Convert to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Display the grayscale image
    display(Image(data=cv2.imencode('.jpg',grayscale_image)[1].tobytes()))
#     # Apply denoising filter
#     denoised_image = cv2.GaussianBlur(grayscale_image, (3, 3), 0)
#     # Apply contrast enhancement
#     equalized_image = cv2.equalizeHist(denoised_image)
#     # Display the preprocessed image
#     display(Image(data=cv2.imencode('.jpg', equalized_image)[1].tobytes()))
#     # Apply image sharpening
#     sharpened_image = cv2.filter2D(equalized_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
#     # Display the preprocessed image
#     display(Image(data=cv2.imencode('.jpg', sharpened_image)[1].tobytes()))
    
    return grayscale_image


  
   

In [6]:
def threshold(image):
    # Apply Otsu thresholding
    _, threshold_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Display the threshold image
    display(Image(data=cv2.imencode('.jpg',threshold_image)[1].tobytes()))
    return threshold_image

In [7]:
def ocrImage(image):
    # Perform OCR on the thresholded image
    recognized_plates=[]
    reader = easyocr.Reader(['en', 'ne'])
    result = reader.readtext(image, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ ', detail=0)
    recognized_plates = [''.join(result)]
    return recognized_plates
    

In [8]:
def recognize_number_plate(image_path,  scale_factor=1):
    vehicle_image = cv2.imread(image_path)
    # Perform object detection using YOLO
    detections = model(vehicle_image)


    # Extract bounding boxes and crop number plate regions
    number_plate_box = None
    for detection in detections[0].boxes.data:
        if detection[5] == 0:  
            number_plate_box = detection[:4]
            break  
            
#      Draw bounding box on the original image
    if number_plate_box is not None:
        x1, y1, x2, y2 = number_plate_box
        cv2.rectangle(vehicle_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Display the original image with bounding box
    display(Image(data=cv2.imencode('.jpg', vehicle_image)[1].tobytes()))
    
    # Crop and Preprocess the number plate region
    if number_plate_box is not None:
        # Crop the number plate region
        cropped_image = vehicle_image[int(y1):int(y2), int(x1):int(x2)]
        # Display the cropped image
        display(Image(data=cv2.imencode('.jpg', cropped_image)[1].tobytes()))
        
         # Scale the cropped image
        scaled_image = cv2.resize(cropped_image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
        # Display the scaled image
        display(Image(data=cv2.imencode('.jpg', scaled_image)[1].tobytes()))
        
        grayscale_image= preprocess_image(scaled_image)
        threshold_image=threshold(grayscale_image)
        ocr= ocrImage(threshold_image)
        
    else:
        print("Number plate not found.")
        
    return ocr


In [9]:
# Path to the image  
# image_path = '/Desktop/final%20project/kk.jpeg'
image_path = 'test_img/123.jpeg'
Image(image_path)
# Recognize the number plate
recognized_plate = recognize_number_plate(image_path)





[ WARN:0@12.758] global loadsave.cpp:244 findDecoder imread_('test_img/123.jpeg'): can't open/read file: check file path/integrity
WARNING ⚠️ 'source' is missing. Using 'source=https://ultralytics.com/images/bus.jpg'.

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /home/rubi/Desktop/final project/Speed-detection-and-number-plate-recognition/numberplate/bus.jpg: 800x608 (no detections), 585.6ms
Speed: 16.7ms preprocess, 585.6ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)


error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1109: error: (-215:Assertion failed) !image.empty() in function 'imencode'


In [ ]:
# Print the recognized number plates
for plate in recognized_plate:
    print(plate)